In [12]:
import os
os.chdir('C:\\Users\\dexte\\sparklyRGT\\sparklyRGT_tutorial')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import scipy.stats as stats
import sparklyRGT as rgt
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows',100)
os.getcwd()

I am being executed!


'C:\\Users\\dexte\\sparklyRGT\\sparklyRGT_tutorial'

In [13]:
os.chdir('C:\\Users\\dexte\\sparklyRGT\\data')
file_names = ['BH06_raw_round1-infusions.xlsx', 'BH06_raw_round1-makeup.xlsx'] 
df = rgt.load_data(file_names)
# rgt.check_sessions(df)

In [15]:
df2 = rgt.edit_groups(df, orig_group = [0], new_group = [3], subs = [5])
df2 = rgt.drop_subjects(df, subs = [7])
df2 = rgt.get_choices(df)
os.chdir('C:\\Users\\dexte\\sparklyRGT\\data')
# file_names = ['BH06_raw_round1-infusions.xlsx']

In [17]:
##will have to write it so that session or group can be passed 
## right now it's going by Group

def prems_after_outcome(df, sessions = 'All'):
    if sessions == 'All':
        sess = df.Group.unique() #all unique sessions
        sess.sort()
    subs = df.Subject.unique() #all unique subjects
    subs.sort()
    df2 = pd.DataFrame(columns = ['Subject','Session','Prems_after_win','Prems_after_loss']) #creates 4 columns
    for sub in subs:
        for num in sess:
            df1 = df.loc[(df.Subject == sub) & (df.Group == num)] #df1 is the raw df for a certain subject and session
            if df1.empty: #if df1 is empty
                df_temp = pd.DataFrame(data = {'Subject': sub, 'Session': num, 'Prems_after_win': np.nan,
                                               'Prems_after_loss': np.nan}, index = [0]) #make one row with NaN 
                df2 = df2.append(df_temp, ignore_index = True) #append that row to df2 
            else:
                prems_after_win = 0
                prems_after_loss = 0
                for i in range(1,len(df1)): #for 2nd trial to rest of trials
                    prev_trial = df1.iloc[i-1, :] #previous trial
                    curr_trial = df1.iloc[i, :] #current trial
                    if curr_trial.Premature_Resp == 1: #if a premature choice occurs...
                        if prev_trial.Rewarded == 1: #if previous trial was a win
                            prems_after_win += 1 #add 1 to prems_after_win
                        if prev_trial.Rewarded == 0 and prev_trial.Premature_Resp == 0: #if previous trial was a loss (and not a premature response)
                            prems_after_loss += 1 #add 1 to prems_after_loss
                df_temp = pd.DataFrame(data = {'Subject': sub, 'Session': num, 'Prems_after_win': prems_after_win,
                                               'Prems_after_loss': prems_after_loss}, index = [0]) #make the row for this subject and session
                df2 = df2.append(df_temp, ignore_index = True) #append this row to df2 
    return df2

pd.set_option('display.max_rows', None)
prem_df = prems_after_outcome(df2) #NaNs are occurring due to df1.empty! could remove 
prem_df

,Subject,Session,Prems_after_win,Prems_after_loss
0,1,1,13,11
1,1,2,11,7
2,1,3,15,12
3,1,4,8,5
4,2,1,9,7
5,2,2,14,4
6,2,3,7,8
7,2,4,8,2
8,3,1,11,6
9,3,2,14,6


In [18]:
#to sum over sessions
for i in range(1,5):
    df3 = prem_df.loc[prem_df['Session']==i]
    print(df3.Prems_after_win.sum(), df3.Prems_after_loss.sum())

143 77
153 80
134 103
154 60
